In [9]:
from dotenv import load_dotenv

load_dotenv()  # This loads the .env file

import os

from agent import create_agent
from prompt import build_restaurant_prompt


graph = create_agent()

graph.get_graph().draw_mermaid_png(output_file_path="graph.png")

# Thread-aware config: persist messages per user/thread for memory
thread_config = {
    "configurable": {
        "thread_id": os.getenv("THREAD_ID", "demo-thread"),
        "user_id": os.getenv("USER_ID", "demo-user"),
    }
}

# Build request from prompt template (flexible by location)
request = build_restaurant_prompt(location="Shinjuku, Tokyo, Japan", top_n=5)

INFO:agent:Creating agent
INFO:agent:Memory created
INFO:agent:Graph created


In [10]:
# Stream with cleaner output while capturing inner state
print("=== Streaming Agent Output ===")
final_state = None
for event in graph.stream(request, config=thread_config):
    if "model" in event:
        msg = event["model"]["messages"]
        try:
            content = getattr(msg, "content", None) or str(msg)
        except Exception:
            content = str(msg)
        if content:
            print(content)
    final_state = event

print("\n=== Final Inner State Snapshot ===")
print(final_state)

=== Streaming Agent Output ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



=== Final Inner State Snapshot ===
{'agent': {'messages': [AIMessage(content="Here are the top 5 restaurants around Shinjuku, Tokyo, showcasing a variety of cuisines and price ranges:\n\n1. **Ichunjipu Shinokubo Korean Restaurant**\n   - **Cuisine:** Korean\n   - **Price:** $$ (Moderate)\n   - **Why it's special:** Highly rated for its authentic Korean dishes and vibrant atmosphere.\n   - [View on Google Maps](https://www.google.com/maps/place/?q=place_id:ChIJlfWxscONGGARpqUGevsCnVo)\n\n2. **Teppan Baby**\n   - **Cuisine:** Japanese (Teppanyaki)\n   - **Price:** $$ (Moderate)\n   - **Why it's special:** Known for its interactive dining experience with fresh ingredients cooked right in front of you.\n   - [View on Google Maps](https://www.google.com/maps/place/?q=place_id:ChIJLZrOfdeMGGARdcjAP55yY00)\n\n3. **Fukumimi Shinjuku**\n   - **Cuisine:** Japanese\n   - **Price:** $$ (Moderate)\n   - **Why it's special:** Offers a cozy atmosphere with a focus on seasonal ingredients and traditi

In [11]:
print(final_state["agent"]["messages"][0].content)

Here are the top 5 restaurants around Shinjuku, Tokyo, showcasing a variety of cuisines and price ranges:

1. **Ichunjipu Shinokubo Korean Restaurant**
   - **Cuisine:** Korean
   - **Price:** $$ (Moderate)
   - **Why it's special:** Highly rated for its authentic Korean dishes and vibrant atmosphere.
   - [View on Google Maps](https://www.google.com/maps/place/?q=place_id:ChIJlfWxscONGGARpqUGevsCnVo)

2. **Teppan Baby**
   - **Cuisine:** Japanese (Teppanyaki)
   - **Price:** $$ (Moderate)
   - **Why it's special:** Known for its interactive dining experience with fresh ingredients cooked right in front of you.
   - [View on Google Maps](https://www.google.com/maps/place/?q=place_id:ChIJLZrOfdeMGGARdcjAP55yY00)

3. **Fukumimi Shinjuku**
   - **Cuisine:** Japanese
   - **Price:** $$ (Moderate)
   - **Why it's special:** Offers a cozy atmosphere with a focus on seasonal ingredients and traditional flavors.
   - [View on Google Maps](https://www.google.com/maps/place/?q=place_id:ChIJVS2On

In [12]:
# import importlib
# from dotenv import load_dotenv

# load_dotenv()  # This loads the .env file

# import tool

# importlib.reload(tool)

# from tool import google_maps_places

# google_maps_places.invoke(
#     {
#         "query": "sushi",
#         "location": "Shinjuku, Tokyo, Japan",
#     }
# )